## Contribution per distance/density to overall visits

In [3]:
import numpy as np

def estimate_visitor_distribution(total_visitors, prob_matrix, distances, densities):
    # Calculate the proportion of total visitors from each density based on their relative values
    density_norm_factor = prob_matrix.sum(axis=0)
    density_norm = densities*density_norm_factor
    density_proportions = density_norm/np.sum(density_norm)
    
    # Calculate the total number of visitors attributed to each density
    visitors_per_density = density_proportions * total_visitors

    # Create an empty array to store the estimated visitors for each distance across all densities
    estimated_visitors_by_distance = np.zeros(len(distances))

    # Distribute each density's visitors across distances according to the probability matrix
    for i in range(len(densities)):
        estimated_visitors_by_distance += prob_matrix[:, i] * visitors_per_density[i]

    # Normalize the results to ensure they sum to the total number of visitors
    estimated_visitors_by_distance *= (total_visitors / np.sum(estimated_visitors_by_distance))

    return estimated_visitors_by_distance, visitors_per_density

# Define the probability matrix, distances, and densities
prob_matrix = np.array([
    [0.999, 0.989, 0.966, 0.906, 0.717, 0.496, 0.236],
    [0.999, 0.979, 0.941, 0.849, 0.610, 0.387, 0.172],
    [0.997, 0.962, 0.899, 0.767, 0.490, 0.288, 0.123],
    [0.995, 0.933, 0.834, 0.659, 0.372, 0.206, 0.086],
    [0.989, 0.883, 0.739, 0.531, 0.268, 0.142, 0.060],
    [0.978, 0.803, 0.615, 0.398, 0.184, 0.096, 0.041]
])
distances = np.array([0, 1, 2, 3, 4, 5])  # in miles
densities = np.array([1, 5, 10, 20, 50, 100, 250])  # thousands of people within a 5-mile radius

total_visitors = 1000000  # total number of visitors
estimated_visitors, visitors_from_each_density = estimate_visitor_distribution(total_visitors, prob_matrix, distances, densities)

print("Estimated Number of Visitors from Each Distance:")
print(estimated_visitors)
print("\nTotal Estimated Visitors by Distance:", np.sum(estimated_visitors))

# Print total visitors from each density
print("\nTotal Visitors from Each Density:")
for density, visitors in zip(densities, visitors_from_each_density):
    print(f"Density {density}k: {visitors:.0f} visitors")

print("\nTotal Estimated Visitors by Density:", np.sum(visitors_from_each_density))


Estimated Number of Visitors from Each Distance:
[246349.32784016 213723.67207287 180755.30112423 148806.53719399
 118825.60543717  91539.55633158]

Total Estimated Visitors by Distance: 1000000.0

Total Visitors from Each Density:
Density 1k: 9324 visitors
Density 5k: 43427 visitors
Density 10k: 78167 visitors
Density 20k: 128660 visitors
Density 50k: 206686 visitors
Density 100k: 252781 visitors
Density 250k: 280955 visitors

Total Estimated Visitors by Density: 1000000.0000000001


## Scaling expenditure per consumer by square footage of store type

In [4]:
total_weekly_visits = 167*1e6
total_annual_visits = 167*1e6 * 52
total_net_sales = 393247*1e6 
avg_spend = (393247*1e6) / (8684*1e6)


## Monte Carlo Visitors + Average Spend

In [5]:
def estimate_visitor_distribution_by_store_type(total_visitors, prob_matrix, distances, densities, store_data, num_simulations=1000):
    # Calculate the proportion of total visitors from each density based on their relative values
    # density_norm_factor = prob_matrix.sum(axis=0)
    # density_norm = densities * density_norm_factor
    # density_proportions = density_norm / np.sum(density_norm)

    # Calculate the total number of visitors attributed to each density
    visitor_results=[]
    for _ in range(num_simulations):
        # Randomly adjust probabilities by up to ±10% to simulate changes in demographics or economic conditions
        adjustment_factor = 1 + (np.random.rand(*prob_matrix.shape) - 0.5) * 0.2
        adjusted_matrix = prob_matrix * adjustment_factor
        density_norm_factor = adjusted_matrix.sum(axis=0)
        density_norm = densities * density_norm_factor
        density_proportions = density_norm / np.sum(density_norm)
        
        visitors_per_density = density_proportions * total_visitors

    # Calculate total visitors based on store type and density eligibility
        total_visitors_by_type = {}     
        for store_type, data in store_data.items():
            # Adjust the total visitors according to the store type specifics
            footprint_ratio = data['avg_sqft'] / np.mean([d['avg_sqft'] for d in store_data.values()])
            location_factor = data['locations'] / sum(d['locations'] for d in store_data.values())
    
            # Apply density eligibility
            eligible_visitors_per_density = visitors_per_density * data['density_eligibility']
            eligible_total_visitors = eligible_visitors_per_density.sum()  # Sum of eligible visitors per density
    
            # Calculate the share of visitors for this store type
            raw_share = eligible_total_visitors * footprint_ratio * location_factor
            total_visitors_by_type[store_type] = raw_share
       
    
        # Normalize the visitor distribution to match the total number of visitors
        sum_raw_shares = sum(total_visitors_by_type.values())
        normalization_factor = total_visitors / sum_raw_shares
        for store_type in total_visitors_by_type:
            total_visitors_by_type[store_type] *= normalization_factor
        
        visitor_results.append(list(total_visitors_by_type.values()))

        # Calculate and distribute visitors
        estimated_visitors_by_distance = np.zeros(len(distances))
        for store_type, visitors in total_visitors_by_type.items():
            #visitor_results.append({store_type:visitors})
            for i, is_eligible in enumerate(store_data[store_type]['density_eligibility']):
                if is_eligible:
                    estimated_visitors_by_distance += prob_matrix[:, i] * (eligible_visitors_per_density[i] * normalization_factor)

    return estimated_visitors_by_distance, visitors_per_density, total_visitors_by_type, visitor_results

# Store-specific data with density eligibility
store_data = {
    'Supercenters': {'locations': 3573, 'avg_sqft': 178000, 'density_eligibility': np.array([1, .8, .8, .8, 0.3, 0, 0])},
    'Discount Stores': {'locations': 370, 'avg_sqft': 105000, 'density_eligibility': np.array([0, .2, .2, .2, 0, 0, 0])},
    'Neighborhood Markets': {'locations': 799, 'avg_sqft': 42000, 'density_eligibility': np.array([0, 0, 0, 0, 0.7, 1, 1])}
}

estimated_visitors, visitors_from_each_density, visitors_by_store_type, visitor_results = estimate_visitor_distribution_by_store_type(
    total_visitors=total_annual_visits, prob_matrix=prob_matrix, distances=distances, densities=densities, store_data=store_data)

print("Estimated Number of Visitors from Each Distance:")
print(estimated_visitors)
print("\nTotal Estimated Visitors by Distance:", np.sum(estimated_visitors))

# Print total visitors from each density
print("\nTotal Visitors from Each Density:")
for density, visitors in zip(densities, visitors_from_each_density):
    print(f"Density {density}k: {visitors:.0f} visitors")

# Print visitors by store type
print("\nVisitors by Store Type:")
for store_type, visitors in visitors_by_store_type.items():
    print(f"{store_type}: {visitors:.0f} visitors")

# Print visitors by store type
print("\nVisitors by Store Type:")

visitors_per_store_by_type = {}
for store_type, visitors in visitors_by_store_type.items():
    visitors_per_store = visitors / store_data[store_type]['locations']
    print(f"{store_type}: {visitors:.0f} total visitors, {visitors_per_store:.0f} visitors per store")
    visitors_per_store_by_type[store_type] = visitors_per_store

print("\nTotal Estimated Visitors by Store Type:", sum(visitors_by_store_type.values()))

print("\nTotal Estimated Visitors by Density:", np.sum(visitors_from_each_density)) 
visitor_results = np.array(visitor_results)
#visitor_results[:,1]
# Analyze results
print("Average Estimated Visitors:", np.mean(visitor_results[:,0]),np.mean(visitor_results[:,1]),np.mean(visitor_results[:,2]))
print("Standard Deviation of Visitors:", np.std(visitor_results[:,0]),np.std(visitor_results[:,1]),np.std(visitor_results[:,2]))
visitors_by_store_type

Estimated Number of Visitors from Each Distance:
[9.00912970e+09 7.27495423e+09 5.61285282e+09 4.14003454e+09
 2.93283374e+09 2.00370283e+09]

Total Estimated Visitors by Distance: 30973507867.07172

Total Visitors from Each Density:
Density 1k: 79052142 visitors
Density 5k: 368617053 visitors
Density 10k: 665287725 visitors
Density 20k: 1131733799 visitors
Density 50k: 1749760992 visitors
Density 100k: 2202264618 visitors
Density 250k: 2487283670 visitors

Visitors by Store Type:
Supercenters: 7585025674 visitors
Discount Stores: 85890843 visitors
Neighborhood Markets: 1013083482 visitors

Visitors by Store Type:
Supercenters: 7585025674 total visitors, 2122873 visitors per store
Discount Stores: 85890843 total visitors, 232137 visitors per store
Neighborhood Markets: 1013083482 total visitors, 1267939 visitors per store

Total Estimated Visitors by Store Type: 8684000000.000002

Total Estimated Visitors by Density: 8684000000.0
Average Estimated Visitors: 7595995469.780447 85517845.0

{'Supercenters': 7585025674.349191,
 'Discount Stores': 85890843.15144148,
 'Neighborhood Markets': 1013083482.4993691}

In [6]:
import numpy as np


# Sample data
visitors_data = {
    'Supercenters': {'total_visitors': 7591667114, 'visitors_per_store': 2124732},
    'Discount Stores': {'total_visitors': 84852921, 'visitors_per_store': 229332},
    'Neighborhood Markets': {'total_visitors': 1007479965, 'visitors_per_store': 1260926}
}
total_revenue = total_net_sales  # Example total revenue

# Define base average spending per visitor for each store type, initially unnormalized
base_spending_per_visitor = {
    'Supercenters': avg_spend,  # Hypothetical average spending per visitor
    'Discount Stores': avg_spend,
    'Neighborhood Markets': avg_spend
}

# Define potential variation ranges for scaling factors (e.g., influenced by e-commerce)
scaling_factor_ranges = {
    'Supercenters': (1.2, 1.4),
    'Discount Stores': (.5, 1.5),
    'Neighborhood Markets': (1, 1.7)
}

# Number of simulations
n_simulations = 10000

def monte_carlo_spending_adjustment(visitors_data, base_spending_per_visitor, scaling_factor_ranges, n_simulations, total_revenue):
    results = []

    for _ in range(n_simulations):
        # Apply random scaling factors to the base spending per visitor
        temp_spending = {store_type: base_spending_per_visitor[store_type] * np.random.uniform(*scaling_factor_ranges[store_type])
                         for store_type in base_spending_per_visitor}
        
        # Normalize the spending using your existing function
        normalized_spending, normalized_total, percent_spending = normalize_spending(visitors_data, temp_spending, total_revenue)
        results.append((normalized_spending, normalized_total, percent_spending))
    
    return results

def normalize_spending(visitors_data, base_spending_per_visitor, total_revenue):
    # Calculate initial total spending
    initial_total_spending = sum(base_spending_per_visitor[store_type] * data['total_visitors']
                                 for store_type, data in visitors_data.items())
    
    # Normalization factor to match the total revenue
    normalization_factor = total_revenue / initial_total_spending
    
    # Adjusted spending per visitor
    normalized_spending_per_visitor = {store_type: base_spending_per_visitor[store_type] * normalization_factor
                                       for store_type in base_spending_per_visitor}
    
    normalized_spending_total = sum(normalized_spending_per_visitor[store_type] * data['total_visitors']
                                    for store_type, data in visitors_data.items())

    normalized_spending_per_store = {store_type: normalized_spending_per_visitor[store_type] * data['total_visitors']
                                       for store_type, data in visitors_data.items()}
    
    percent_spending_per_store = {store_type: normalized_spending_per_store[store_type] / normalized_spending_total
                                       for store_type in normalized_spending_per_store}
    
    return normalized_spending_per_visitor, normalized_spending_total, percent_spending_per_store

def calculate_averages(simulation_results):
    # Initialize dictionaries to accumulate totals for averaging later
    total_normalized_spending_per_store = {store_type: 0 for store_type in visitors_data}
    total_percent_spending_per_store = {store_type: 0 for store_type in visitors_data}
    total_normalized_spending_per_visitor = {store_type: 0 for store_type in visitors_data}
    
    # Accumulate totals from each simulation
    for normalized_spending, normalized_total, percent_spending in simulation_results:
        for store_type in visitors_data:
            total_normalized_spending_per_store[store_type] += normalized_spending[store_type] * visitors_data[store_type]['total_visitors']
            total_percent_spending_per_store[store_type] += percent_spending[store_type]
            total_normalized_spending_per_visitor[store_type] += normalized_spending[store_type]
            
    # Calculate averages by dividing by the number of simulations
    avg_normalized_spending_per_store = {store_type: total / n_simulations for store_type, total in total_normalized_spending_per_store.items()}
    avg_percent_spending_per_store = {store_type: total / n_simulations for store_type, total in total_percent_spending_per_store.items()}
    avg_normalized_spending_per_visitor = {store_type: total / n_simulations for store_type, total in total_normalized_spending_per_visitor.items()}
    
    return avg_normalized_spending_per_store, avg_percent_spending_per_store, avg_normalized_spending_per_visitor



# Run the Monte Carlo simulation
simulation_results = monte_carlo_spending_adjustment(visitors_data, base_spending_per_visitor, scaling_factor_ranges, n_simulations, total_revenue)


# Get averages from the simulation results
avg_normalized_spending_per_store, avg_percent_spending_per_store, avg_normalized_spending_per_visitor = calculate_averages(simulation_results)

# Printing the averages
print("Average Normalized Spending per Store:")
for store_type, avg_spending in avg_normalized_spending_per_store.items():
    print(f"{store_type}: ${avg_spending:.2f}")

print("\nAverage Percent Spending by Store Type:")
for store_type, avg_percent in avg_percent_spending_per_store.items():
    print(f"{store_type}: {avg_percent * 100:.2f}%")

print("\nAverage Visitor Spending by Store Type:")
for store_type, avg_spending in avg_normalized_spending_per_visitor.items():
    print(f"{store_type}: ${avg_spending:.2f}")




# Example output
average_results = np.mean([result[1] for result in simulation_results])
print("Average Normalized Spending Total across Simulations:", average_results)


Average Normalized Spending per Store:
Supercenters: $343092498150.90
Discount Stores: $2941537999.41
Neighborhood Markets: $47212963849.69

Average Percent Spending by Store Type:
Supercenters: 87.25%
Discount Stores: 0.75%
Neighborhood Markets: 12.01%

Average Visitor Spending by Store Type:
Supercenters: $45.19
Discount Stores: $34.67
Neighborhood Markets: $46.86
Average Normalized Spending Total across Simulations: 393247000000.0


## The Expansion Simulation Class

In [7]:
densities, prob_matrix

(array([  1,   5,  10,  20,  50, 100, 250]),
 array([[0.999, 0.989, 0.966, 0.906, 0.717, 0.496, 0.236],
        [0.999, 0.979, 0.941, 0.849, 0.61 , 0.387, 0.172],
        [0.997, 0.962, 0.899, 0.767, 0.49 , 0.288, 0.123],
        [0.995, 0.933, 0.834, 0.659, 0.372, 0.206, 0.086],
        [0.989, 0.883, 0.739, 0.531, 0.268, 0.142, 0.06 ],
        [0.978, 0.803, 0.615, 0.398, 0.184, 0.096, 0.041]]))

In [13]:
import numpy as np

class NeighborhoodMarketExpansionModel:
    def __init__(self, demographic_data, market_conditions, financials, investment_factors, historical_data, store_data, prob_matrix, distances, densities, avg_spending_per_visitor):
        self.demographic_data = demographic_data
        self.market_conditions = market_conditions
        self.financials = financials
        self.investment_factors = investment_factors
        self.historical_data = historical_data
        self.store_data = store_data
        self.prob_matrix = prob_matrix
        self.distances = distances
        self.densities = densities
        self.avg_spending_per_visitor = avg_spending_per_visitor
        self.visitor_estimates = None

    
    def calculate_cagr(self, initial_value, final_value, periods):
        if periods <= 0:
            raise ValueError("Periods must be greater than zero for CAGR calculation.")
        return (final_value / initial_value) ** (1 / periods) - 1

    def calculate_cagrs(self):
        revenue_cagr = self.calculate_cagr(
            self.historical_data['initial_revenue'],
            self.historical_data['final_revenue'],
            self.historical_data['periods'])
        cogs_cagr = self.calculate_cagr(
            self.historical_data['initial_cogs'],
            self.historical_data['final_cogs'],
            self.historical_data['periods'])
        ecommerce_cagr = self.calculate_cagr(
            self.historical_data['initial_ecommerce'],
            self.historical_data['final_ecommerce'],
            self.historical_data['periods'])
        return revenue_cagr, cogs_cagr, ecommerce_cagr

    def calculate_cannibalization_effect(self, existing_store_sales, new_store_openings):
        cannibalization_rate = self.market_conditions.get('cannibalization_rate', 0.01)
        adjusted_sales = existing_store_sales * (1 - cannibalization_rate * new_store_openings)
        return adjusted_sales

    def dynamic_ecommerce_growth(self, current_sales):
        ecommerce_growth_factor = self.market_conditions.get('ecommerce_growth_factor', 0.05)
        market_saturation = self.market_conditions.get('market_saturation', 0.1)
        distribution_efficiency = self.market_conditions.get('distribution_efficiency', 0.9)
        adjusted_growth_rate = ecommerce_growth_factor * (1 - market_saturation) * distribution_efficiency
        adjusted_sales = current_sales * (1 + adjusted_growth_rate)
        return adjusted_sales

    def adjust_operational_costs(self, base_cost):
        proximity_factor = self.market_conditions.get('proximity_factor', 0.2)
        distribution_center_proximity = self.market_conditions.get('distribution_center_proximity', 0.5)
        adjusted_cost = base_cost * (1 - proximity_factor * distribution_center_proximity)
        return adjusted_cost

    def labor_investment_needed(self, current_market_share, target_market_share, community_opposition_strength):
        labor_investment_increase = 0
        while current_market_share < target_market_share:
            current_market_share *= (1 + 0.01 * (1 - community_opposition_strength))
            labor_investment_increase += 0.01
        return labor_investment_increase

    def evaluate_labor_investment_impact(self, labor_costs, initial_profit_margin, labor_investment_increase):
        adjusted_labor_costs = labor_costs * (1 + labor_investment_increase)
        adjusted_profit_margin = initial_profit_margin * (1 - 0.05 * labor_investment_increase)
        return adjusted_labor_costs, adjusted_profit_margin


    def estimate_visitors(self, total_visitors):
        # Call the visitor distribution estimation function
        estimated_visitors, visitors_from_each_density, visitors_by_store_type, visitor_results = estimate_visitor_distribution_by_store_type(
            total_visitors=total_visitors, prob_matrix=self.prob_matrix, distances=self.distances, densities=self.densities, store_data=self.store_data)
        self.visitor_estimates = {
            'estimated_visitors': estimated_visitors,
            'visitors_from_each_density': visitors_from_each_density,
            'visitors_by_store_type': visitors_by_store_type,
            'visitor_results': visitor_results
        }
        return self.visitor_estimates

    def simulate_store_numbers(visitors_by_store_type, optimal_visitors, max_stores, store_type):
        results = {}
        for num_stores in range(1, max_stores + 1):
            visitors_per_store = visitors_by_store_type[store_type] / (store_data[store_type]['locations'] + num_stores)
            if visitors_per_store < optimal_visitors:
                # Assuming profitability drops if the store is underutilized
                profitability = visitors_per_store / optimal_visitors * 100  # as percentage
            else:
                # Assuming maximum profitability is reached at optimal visitors per store
                profitability = 100
    
            results[num_stores] = (visitors_per_store, profitability)
            if visitors_per_store < optimal_visitors:
                break  # Stop if stores are not reaching optimal visitor count
    
        return results

    def run_monte_carlo_simulations(self, num_simulations=1000, total_visitors=167000000):
        # Estimate visitor distribution
        self.estimate_visitors(total_visitors)

        results = []
        for _ in range(num_simulations):
            # Example of utilizing visitor counts
            for store_type, visitors in self.visitor_estimates['visitors_by_store_type'].items():
                current_sales = visitors * self.avg_spending_per_visitor.get(store_type, 20)  # Assuming a default average spend if not specified
                adjusted_sales = self.calculate_cannibalization_effect(current_sales, np.random.randint(1, 10))
                adjusted_sales = self.dynamic_ecommerce_growth(adjusted_sales)
                adjusted_costs = self.adjust_operational_costs(self.financials['base_operational_cost'])
                net_profit = adjusted_sales - adjusted_costs
                results.append(net_profit)

        average_profit = np.mean(results)
        profit_std_dev = np.std(results)
        return average_profit, profit_std_dev





In [16]:
estimated_visitors, visitors_from_each_density, visitors_by_store_type, visitor_results = estimate_visitor_distribution_by_store_type(
total_visitors=total_annual_visits, prob_matrix=prob_matrix, distances=distances, densities=densities, store_data=store_data)
print(visitors_by_store_type)
def simulate_store_numbers(visitors_by_store_type, optimal_visitors, max_stores, store_type):
    results = {}
    for num_stores in range(1, max_stores + 1):
        visitors_per_store = visitors_by_store_type[store_type] / (store_data[store_type]['locations'] + num_stores)
        if visitors_per_store < optimal_visitors:
            # Assuming profitability drops if the store is underutilized
            profitability = visitors_per_store / optimal_visitors * 100  # as percentage
        else:
            # Assuming maximum profitability is reached at optimal visitors per store
            profitability = 100

        results[num_stores] = (visitors_per_store, profitability)
        if visitors_per_store < optimal_visitors:
            break  # Stop if stores are not reaching optimal visitor count

    return results


# Optimal visitors per store for efficient operation
optimal_visitors_per_store = 1200000


max_possible_stores = 50  # Set based on your market size and investment capability
simulation_results = simulate_store_numbers(visitors_by_store_type, optimal_visitors_per_store, max_possible_stores, 'Neighborhood Markets')

simulation_results


{'Supercenters': 7629368223.17538, 'Discount Stores': 86962038.30518955, 'Neighborhood Markets': 967669738.5194305}


{1: (1209587.1731492882, 100),
 2: (1208077.076803284, 100),
 3: (1206570.7462835792, 100),
 4: (1205068.1675210842, 100),
 5: (1203569.3265167046, 100),
 6: (1202074.2093409074, 100),
 7: (1200582.8021332885, 100),
 8: (1199095.0911021444, 99.9245909251787)}

In [17]:
prob_matrix = np.array([
    [0.999, 0.989, 0.966, 0.906, 0.717, 0.496, 0.236],
    [0.999, 0.979, 0.941, 0.849, 0.610, 0.387, 0.172],
    [0.997, 0.962, 0.899, 0.767, 0.490, 0.288, 0.123],
    [0.995, 0.933, 0.834, 0.659, 0.372, 0.206, 0.086],
    [0.989, 0.883, 0.739, 0.531, 0.268, 0.142, 0.060],
    [0.978, 0.803, 0.615, 0.398, 0.184, 0.096, 0.041]
])
distances = np.array([0, 1, 2, 3, 4, 5])  # in miles
densities = np.array([1, 5, 10, 20, 50, 100, 250])  # thousands of people within a 5-mile radius




# Example usage of the class:
model = NeighborhoodMarketExpansionModel(
    demographic_data={},
    market_conditions={"cannibalization_rate": 0.01, "ecommerce_growth_factor": 0.05},
    financials={ "base_operational_cost": 20000, "investment": 100000},
    investment_factors={},
    historical_data={},
    prob_matrix=prob_matrix,
    densities=densities,
    store_data = {
    'Supercenters': {'locations': 3573, 'avg_sqft': 178000, 'density_eligibility': np.array([1, .8, .8, .8, 0.3, 0, 0])},
    'Discount Stores': {'locations': 370, 'avg_sqft': 105000, 'density_eligibility': np.array([0, .2, .2, .2, 0, 0, 0])},
    'Neighborhood Markets': {'locations': 799, 'avg_sqft': 42000, 'density_eligibility': np.array([0, 0, 0, 0, 0.7, 1, 1])}
},
    distances=distances,
    avg_spending_per_visitor = {'Supercenters': 35.0, 'Discount Stores': 28.5, 'Neighborhood Markets': 32.0}
)
cann=model.calculate_cannibalization_effect(1000927994.9299742,9)
average_profit, profit_std_dev = model.run_monte_carlo_simulations()
print(f"Average Profit from Simulations: {average_profit}")
print(f"Profit Standard Deviation: {profit_std_dev}")




Average Profit from Simulations: 1903964963.980138
Profit Standard Deviation: 2236527816.3008447


In [ ]:
import numpy as np

class NeighborhoodMarketExpansionModel:
    def __init__(self, market_conditions, financials, investment_factors, historical_data, store_data, prob_matrix, distances, densities, avg_spending_per_visitor, financial_forecast):
        # self.demographic_data = demographic_data
        self.market_conditions = market_conditions
        self.financials = financials
        self.investment_factors = investment_factors
        self.historical_data = historical_data
        self.store_data = store_data
        self.prob_matrix = prob_matrix
        self.distances = distances
        self.densities = densities
        self.avg_spending_per_visitor = avg_spending_per_visitor
        self.financial_forecast = financial_forecast  # Baseline financial data
        self.visitor_estimates = None

    # Existing methods...

    def update_financial_forecasts(self, num_new_stores, store_type):
        # Placeholder for updating financial forecasts based on new stores
        # This would typically modify self.financial_forecast based on estimated additional revenue, costs, and asset changes
        revenue_increase = num_new_stores * self.estimate_additional_revenue_per_store(store_type)
        new_assets_value = num_new_stores * self.store_data[store_type]['average_asset_value']
        self.financial_forecast['Operating revenue (Turnover)'] += revenue_increase
        self.financial_forecast['Total assets'] += new_assets_value
        # More adjustments can be added here

    def estimate_additional_revenue_per_store(self, store_type):
        # Estimate additional revenue per store - this could use historical data or market analysis
        return 20000  # Example fixed value

    def simulate_store_impact(self, max_new_stores, store_type):
        # Simulate financial impact for adding up to max_new_stores
        for num_stores in range(1, max_new_stores + 1):
            self.update_financial_forecasts(num_stores, store_type)
            print(f"After adding {num_stores} {store_type} stores: Updated Financial Forecast")


market_conditions={'community_opposition': 0.5, 'cannibalization_rate': 0.01, 'ecommerce_growth_factor': 0.03}
# Example instantiation and usage:
model = NeighborhoodMarketExpansionModel(market_conditions, financials, investment_factors, historical_data, store_data, prob_matrix, distances, densities, avg_spending_per_visitor, financial_forecast)
model.simulate_store_impact(5, 'Neighborhood markets')


## Trying to workout expenses

## Additional insights from appendices

In [ ]:
class WalmartFinancialModel:
    # Existing methods...

    def calculate_supply_chain_efficiencies(self, automation_investments, year):
        # Estimate cost savings and revenue impacts from supply chain automation
        savings = automation_investments * 0.05  # Assuming 5% cost saving from automation
        additional_revenue = savings * 3  # Hypothetical multiplier for revenue from improved delivery speeds
        return savings, additional_revenue
    
    def evaluate_membership_programs(self, subscribers, subscription_fee):
        # Calculate revenue from subscription-based models like Walmart+
        return subscribers * subscription_fee

    def scenario_analysis(self, base_case_parameters, scenarios):
        # Conduct scenario analysis based on varying parameters
        results = {}
        for scenario, params in scenarios.items():
            self.update_market_conditions(params)  # Method to dynamically adjust market conditions
            results[scenario] = self.run_financial_forecast()
        return results


In [ ]:
class SupplyChainModel:
    def __init__(self, initial_investment, efficiency_rate, revenue_growth_rate,distribution_sqft_demand):
        self.initial_investment = initial_investment
        self.efficiency_rate = cost_savings_rate  # Percentage of operational cost savings
        self.revenue_growth_rate = revenue_growth_rate  # Additional revenue growth from enhanced capabilities
        self.distribution_sqft_demand

    def calculate_costs(self, operational_costs):
        # Calculate cost savings based on the initial investment and operational cost savings rate
        costs = { coef	std err	t	P>|t|	[0.025	0.975]
Discount stores	12.5885	1.454	8.660	0.000	9.588	15.589
Supercenters	16.2044	3.715	4.361	0.000	8.536	23.873
Neighborhood markets	26.8763	9.679	2.777	0.010	6.899	46.854
Capital Expenditures	-3.1922	0.655	-4.872	0.000	-4.545	-1.840} 
        savings = operational_costs * self.cost_savings_rate
        return savings

    def calculate_revenue_growth(self, base_revenue):
        # Calculate additional revenue generated from investments
        additional_revenue = base_revenue * self.revenue_growth_rate
        return additional_revenue

    def simulate_financial_impact(self, operational_costs, base_revenue):
        # Calculate both cost savings and additional revenue
        savings = self.calculate_cost_savings(operational_costs)
        additional_revenue = self.calculate_revenue_growth(base_revenue)
        
        # Net effect on operations
        net_operational_costs = operational_costs - savings
        net_revenue = base_revenue + additional_revenue

        return net_operational_costs, net_revenue

# # Example usage
# initial_investment = 50000000  # $50 million
# operational_costs = 100000000  # $100 million annual operational costs
# base_revenue = 500000000  # $500 million base revenue

# supply_chain_model = SupplyChainModel(initial_investment, cost_savings_rate=0.1, revenue_growth_rate=0.05)
# net_costs, net_revenue = supply_chain_model.simulate_financial_impact(operational_costs, base_revenue)

# print(f"Net Operational Costs after Savings: ${net_costs}")
# print(f"Net Revenue after Investment: ${net_revenue}")


In [ ]:
class ECommerceGrowth:
    def __init__(self, base_sales, growth_rate, subscribers, subscription_fee, fulfillment_impact):
        self.base_sales = base_sales
        self.growth_rate = growth_rate
        self.subscribers = subscribers
        self.subscription_fee = subscription_fee
        self.fulfillment_impact = fulfillment_impact

    def calculate_incremental_revenue_from_subscriptions(self):
        # Direct revenue from subscriptions
        direct_revenue = self.subscribers * self.subscription_fee
        
        # Indirect revenue could be modeled based on increased purchase frequency
        # Assuming a 5% increase in overall purchases due to membership benefits
        indirect_revenue = self.base_sales * 0.05
        return direct_revenue + indirect_revenue

    def calculate_revenue_from_fulfillment_expansion(self):
        # Additional revenue from enhanced fulfillment capabilities
        # Assuming a 3% increase in sales due to better customer service and faster delivery
        additional_revenue = self.base_sales * self.fulfillment_impact
        return additional_revenue

    def simulate_e_commerce_growth(self):
        # Calculate total e-commerce growth including Walmart+ and fulfillment impacts
        subscription_revenue = self.calculate_incremental_revenue_from_subscriptions()
        fulfillment_revenue = self.calculate_revenue_from_fulfillment_expansion()
        total_revenue = self.base_sales + subscription_revenue + fulfillment_revenue
        return total_revenue

# # Example usage
# ecommerce_growth = ECommerceGrowth(base_sales=100000000, growth_rate=0.1, subscribers=1000000, subscription_fee=98, fulfillment_impact=0.03)
# total_revenue = ecommerce_growth.simulate_e_commerce_growth()

# print(f"Total Projected E-Commerce Revenue: ${total_revenue}")


In [ ]:
# Modify the SupplyChainModel to focus exclusively on cost savings
class SupplyChainModel:
    def __init__(self, initial_investment, cost_savings_rate):
        self.initial_investment = initial_investment
        self.cost_savings_rate = cost_savings_rate  # Only focus on cost savings

    def calculate_cost_savings(self, operational_costs):
        # Calculate cost savings based on operational efficiencies
        return operational_costs * self.cost_savings_rate

# Redefine the ECommerceGrowth to explicitly cover all revenue impacts, removing generic revenue growth from supply chain improvements
class ECommerceGrowth:
    def __init__(self, base_sales, subscribers, subscription_fee, fulfillment_impact):
        self.base_sales = base_sales
        self.subscribers = subscribers
        self.subscription_fee = subscription_fee
        self.fulfillment_impact = fulfillment_impact  # Include all revenue enhancements from fulfillment

    def simulate_e_commerce_growth(self):
        # Calculate direct revenue from subscriptions
        direct_revenue = self.subscribers * self.subscription_fee
        # Revenue from enhanced fulfillment capabilities
        additional_revenue = self.base_sales * (0.05 + self.fulfillment_impact)  # Include indirect revenue from subscriptions and fulfillment improvements
        return self.base_sales + direct_revenue + additional_revenue

# This adjustment ensures that the revenue from supply chain improvements is not double-counted with the fulfillment enhancements in ECommerceGrowth.


In [ ]:
import numpy as np

class FinancialModel:
    def __init__(self, base_sales, operational_costs, supply_chain_params, ecommerce_params):
        self.base_sales = base_sales
        self.operational_costs = operational_costs
        self.supply_chain_model = SupplyChainModel(**supply_chain_params)
        self.ecommerce_growth = ECommerceGrowth(**ecommerce_params)

    def run_simulation(self, num_simulations=1000):
        results = []
        for _ in range(num_simulations):
            # Randomly generate input parameters from assumed distributions
            sampled_operational_costs = np.random.normal(self.operational_costs, self.operational_costs * 0.1)  # 10% SD
            sampled_base_sales = np.random.normal(self.base_sales, self.base_sales * 0.1)  # 10% SD

            # Calculate impacts from each model component
            net_operational_costs = self.supply_chain_model.calculate_cost_savings(sampled_operational_costs)
            net_revenue = self.ecommerce_growth.simulate_e_commerce_growth()

            # Calculate net profit or other financial metrics
            net_profit = net_revenue - net_operational_costs
            results.append(net_profit)

        return np.mean(results), np.std(results)  # Return average and standard deviation of results

# Example usage with some hypothetical parameters
params = {
    'base_sales': 500000000,
    'operational_costs': 100000000,
    'supply_chain_params': {'initial_investment': 50000000, 'cost_savings_rate': 0.05},
    'ecommerce_params': {'base_sales': 500000000, 'subscribers': 1000000, 'subscription_fee': 98, 'fulfillment_impact': 0.03}
}
model = FinancialModel(**params)
average_profit, profit_std_dev = model.run_simulation(num_simulations=10000)

print(f"Average Projected Profit: ${average_profit:.2f}")
print(f"Profit Standard Deviation: ${profit_std_dev:.2f}")
